THIS NOTEBOOK NEEDS TO BE REVISITED AND UPDATED. IGNORE FOR NOW.

In [3]:
import dominoes

# Tabular Q-Learning for dominoes
in this notebook, we'll be developing a RL agent inside of the dominoes library.


---
**Notes to self**
- [ ] trouble with move proposal


# 1. Observing the environment

for now, we'll assume player 0 always starts. In the following block, a game is initialized and its state is displayed. We assume each player only knows her hand

In [5]:
#agent with be player 0
game = dominoes.Game.new(starting_domino = dominoes.Domino(6, 6) )

game

Board: [6|6]
Player 0's hand: [0|4][0|3][2|4][2|6][4|6][3|6][3|4]
Player 1's hand: [2|2][2|5][1|6][0|5][0|1][1|1][4|5]
Player 2's hand: [5|6][0|0][1|4][3|3][0|2][1|3]
Player 3's hand: [4|4][5|5][1|5][0|6][3|5][2|3][1|2]
Player 3's turn

now, we display i) the state of the board and ii) the agents hands

In [65]:
game.board, game.hands[0] # state of the game

([6|6], [0|5][2|6][2|2][1|1][1|6][1|3][2|3])

In [27]:
turn = game.turn; turn # indicates which player is next (0-3)

1

In [38]:
game.turn, game.hands[game.turn]

(2, [5|5][1|3][3|3][0|2][3|4][4|5][2|4])

### building a simple state representation

In [39]:
def simple_state(game):
    return game.board.left_end(), game.board.right_end(), game.turn, game.hands



In [66]:
simple_state(game)

(6,
 6,
 2,
 [[0|5][2|6][2|2][1|1][1|6][1|3][2|3],
  [0|0][2|4][2|5][4|6][4|4][3|6],
  [3|4][0|6][5|5][0|4][1|5][5|6][0|2],
  [0|1][3|5][1|4][1|2][0|3][3|3][4|5]])

In [47]:
d = dominoes.Domino(6,3); d.second


3

In [52]:
game

Board: [0|6][6|6]
Player 0's hand: [0|1][2|2][0|0][0|4][2|6][2|5]
Player 1's hand: [0|3][1|1][1|5][4|4][0|5][3|5]
Player 2's hand: [5|5][1|3][3|3][0|2][3|4][4|5][2|4]
Player 3's hand: [1|2][1|4][2|3][5|6][3|6][4|6][1|6]
Player 2's turn

In [63]:
def available_actions(state):
    """
    Important function! should implement as method.
    uses game state to return list of available actions for the current
    player
    """
    left, right, turn, hands = state
    actions = set()
    for domino in hands[turn]:
        if left in domino:
            actions.add((domino, True))
        if right in domino:
            actions.add((domino, False))
    return actions

In [62]:
state = simple_state(game)
available_actions(state)

{([0|2], True)}

In [ ]:
# example move: ()

# 2. Executing an action

here we take a look at exactly how an agent can take an action that influences the board.



The following method provides the a tuple of the current player's dominoes and a boolean indicating whether it can be played or not. The automated players that have already been implemented basically just reorder this tuple of tuples. 

---
I have some questions here:
- Why is this done this way? could I do it another way?
- what exactly does the valid_moves method do? 
 - *it contains all of the moves in the hand of the current player. I don't really like how its implemented (its name is misleading).*


In [30]:
# before applying the player [ranking] algorithm
game.valid_moves

(([0|6], True),)

In [31]:
game.make_move(*game.valid_moves[0])

In [32]:
game

Board: [0|6][6|6]
Player 0's hand: [0|1][2|2][0|0][0|4][2|6][2|5]
Player 1's hand: [0|3][1|1][1|5][4|4][0|5][3|5]
Player 2's hand: [5|5][1|3][3|3][0|2][3|4][4|5][2|4]
Player 3's hand: [1|2][1|4][2|3][5|6][3|6][4|6][1|6]
Player 2's turn

In [33]:
game.valid_moves

(([0|2], True),)

now we take a look at how some of the automated players produce there moves. In particular, we notice that when the player is called on the game object, it restructures the current player's hand according to its preference.

In [13]:

# automated players prefers to play highest domino.
bota_gorda = dominoes.players.bota_gorda

# apply the player setting to select a
# move to play. the player setting is a callable that will sort the
# game's valid moves in decreasing order of preference.
bota_gorda(game) # can see them as game.valid_moves

the new ranking of the player's hand is now different....

In [24]:
# after applying the player [ranking] algorithm
game.valid_moves

(([4|6], True), ([1|6], True))

In [48]:
game.valid_moves[0] 

# second value in tuple is the side to play. True is left, False is right.


([6|6], True)

In [49]:
# make the selected move

game.make_move(*game.valid_moves[0]) # why the asterisk??? 
# asterisk argument => play the first move that is playable according to the 
# game rules. basically loops through the entire list.

In [23]:
game

Board: [6|6]
Player 0's hand: [3|6][1|1][5|5][1|5][1|3][1|2]
Player 1's hand: [2|4][2|3][2|5][1|4][4|6][1|6][0|4]
Player 2's hand: [0|1][3|5][5|6][4|5][0|5][3|4][0|3]
Player 3's hand: [2|2][3|3][0|2][4|4][0|0][2|6][0|6]
Player 1's turn

# 3. Building an agent

- [X] determining the reward function
- [ ] epsilon-greedy exploration algorithm
- [ ] Q- learning

## i.  Rewards

Reward schedule:

- Win: +points earned
- Game continues: 0
- Lose: -points earned




In [51]:
def reward(game):
    
    # game hasn't finished
    if game.result is not None:
        # if agent teams wins, reward = points earned
        if game.result.player == 0 or game.result.player == 2:
            # return 1
            return game.result.points
        # if agent teams wins, reward = -( points lost)
        else:
            # return -1
            return -game.result.points
    
    # if game hasn't finished, do not return
    return 0
        
    

In [ ]:
"""
var points: the absolute value of this quantity indicates
             the amount of points earned by the winning team.
             This quantity is positive if the team consisting
             of players 0 and 2 won, negative if the team
             consisting of players 1 and 3 won, and 0 in case
             of a tie.
"""


In [52]:
reward(game)

0

## ii. e-greedy algorithm

In [53]:
### code obtained from 
### https://gymnasium.farama.org/environments/toy_text/frozen_lake/

 ### ---- will comment out algorithm for now
    
#     self.q_values = defaultdict(lambda: np.zeros(env.action_space.n))

#     def get_action(self, obs: tuple[int, int, bool]) -> int:
        
#         """
#         Returns the best action with probability (1 - epsilon)
#         otherwise a random action with probability epsilon to ensure exploration.
#         """
#         # with probability epsilon return a random action to explore the environment
#         if np.random.random() < self.epsilon:
#             return env.action_space.sample()

#         # with probability (1 - epsilon) act greedily (exploit)
#         else:
#             return int(np.argmax(self.q_values[obs]))

#         ...
        

IndentationError: unexpected indent (3269743613.py, line 4)

## iii. Q-learning

- `self.q_values`: This is a dictionary that will store Q-values for each state-action pair in a reinforcement learning problem. The keys of this dictionary represent states, and the values are arrays representing the Q-values for each possible action in that state.

- `defaultdict`: defaultdict is a type of dictionary provided by the collections module in Python. Here we're using defaultdict to ensure that if you access a state that hasn't been seen before, it will automatically create a new entry for that state with a default value.


In [57]:
# code obtained from 
# https://gymnasium.farama.org/environments/toy_text/frozen_lake/

self.q_values = defaultdict(lambda: np.zeros(env.action_space.n))

NameError: name 'self' is not defined